In [ ]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [ ]:
import folium
import numpy as np
import os
import pandas as pd
import pickle

In [ ]:
def visualize(filename, path_to_the_file_with_time, data_gps):
    
    with open(os.path.join("models", "encoder.pcl"), "rb") as file:
        encoder = pickle.load(file)
    
    data_time = pd.read_csv(path_to_the_file_with_time)
    
    data_pred = pd.read_csv(filename)
    data_pred["time"] = data_time['time']
    
    unique, counts = np.unique(data_pred['anomalies_category'].values, return_counts=True)
    
    print(encoder.inverse_transform(unique), counts)
    data_pred['anomalies_category'] = encoder.inverse_transform(data_pred['anomalies_category'])
    def concat_with_event(row):
        data = data_pred[((data_pred["time"] >= row['time'] - 10) & (data_pred["time"] <= row['time'] + 10))].reset_index()
        try:
            return data['anomalies_category'].values[0]
        except:
            return np.nan
    
    data_gps['anomalies_category'] = data_gps.apply(concat_with_event, axis=1)
    data_gps = data_gps.dropna()
    data_gps['speed'] = data_gps['speed'] * 3.6
    data_gps['speed'] = data_gps['speed'].apply(round)
    coords_critical = data_gps[['lat', 'lng', "speed"]][data_gps['anomalies_category']=='critical'].values.tolist()
    coords_negligible = data_gps[['lat', 'lng', "speed"]][data_gps['anomalies_category']=='negligible'].values.tolist()
    coords_significant = data_gps[['lat', 'lng', "speed"]][data_gps['anomalies_category']=='significant'].values.tolist()
    mapit = folium.Map(location=[data_gps['lat'].min(), data_gps['lng'].min()], zoom_start=15)

    for coord in coords_negligible:
        folium.Marker(location=[coord[0], coord[1]], icon = folium.DivIcon(html=f"""<div style="color: green;">{coord[2]}</div>""")).add_to(mapit)

    for coord in coords_significant:
        folium.Marker(location=[coord[0], coord[1]], icon = folium.DivIcon(html=f"""<div style="color: black;">{coord[2]}</div>""")).add_to(mapit)

    for coord in coords_critical:
        folium.Marker(location=[coord[0], coord[1]], icon = folium.DivIcon(html=f"""<div style="color: red;">{coord[2]}</div>""")).add_to(mapit)
    
    return mapit

In [ ]:
data_gps = pd.read_csv(os.path.join('data', 'test', 'sensorlab_2020-02-15-06.52.25', 'gps.csv'))
filename = 'sensorlab_2020-02-15-06.52.25_accelerometer.csv'
path_to_the_file_with_time = os.path.join('data', 'test', 'sensorlab_2020-02-15-06.52.25', "accelerometer.csv")
visualize(os.path.join('data', 'test', 'predict', 'linear', filename), path_to_the_file_with_time, data_gps)

In [ ]:
filename = 'sensorlab_2020-02-15-06.52.25_accelerometer_gyroscope.csv'
path_to_the_file_with_time = os.path.join('data', 'test', 'sensorlab_2020-02-15-06.52.25', "accelerometer.csv")
visualize(os.path.join('data', 'test', 'predict', 'linear', filename), path_to_the_file_with_time, data_gps)

In [ ]:
filename = 'sensorlab_2020-02-15-06.52.25_accelerometer.csv'
path_to_the_file_with_time = os.path.join('data', 'test', 'sensorlab_2020-02-15-06.52.25', "accelerometer.csv")
visualize(os.path.join('data', 'test', 'predict', 'non-linear', filename), path_to_the_file_with_time, data_gps)

In [ ]:
filename = 'sensorlab_2020-02-15-06.52.25_accelerometer_gyroscope.csv'
path_to_the_file_with_time = os.path.join('data', 'test', 'sensorlab_2020-02-15-06.52.25', "accelerometer.csv")
visualize(os.path.join('data', 'test', 'predict', 'non-linear', filename), path_to_the_file_with_time, data_gps)

In [ ]:
filename = 'sensorlab_2020-02-15-06.52.25_accelerometer_features.csv'
path_to_the_file_with_time = os.path.join('data', 'test', 'sensorlab_2020-02-15-06.52.25', "accelerometer.csv")
visualize(os.path.join('data', 'test', 'predict', 'linear-features', filename), path_to_the_file_with_time, data_gps)

In [ ]:
filename = 'sensorlab_2020-02-15-06.52.25_accelerometer_gyroscope_features.csv'
path_to_the_file_with_time = os.path.join('data', 'test', 'sensorlab_2020-02-15-06.52.25', "accelerometer.csv")
visualize(os.path.join('data', 'test', 'predict', 'linear-features', filename), path_to_the_file_with_time, data_gps)

In [ ]:
filename = 'sensorlab_2020-02-15-06.52.25_accelerometer_features.csv'
path_to_the_file_with_time = os.path.join('data', 'test', 'sensorlab_2020-02-15-06.52.25', "accelerometer.csv")
visualize(os.path.join('data', 'test', 'predict', 'non-linear-features', filename), path_to_the_file_with_time, data_gps)

In [ ]:
filename = 'sensorlab_2020-02-15-06.52.25_accelerometer_gyroscope_features.csv'
path_to_the_file_with_time = os.path.join('data', 'test', 'sensorlab_2020-02-15-06.52.25', "accelerometer.csv")
visualize(os.path.join('data', 'test', 'predict', 'non-linear-features', filename), path_to_the_file_with_time, data_gps)